In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from xgboost.sklearn import XGBClassifier
from pandas.plotting import scatter_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier,BaggingClassifier,VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

In [153]:
dataset = pd.read_csv('G:\dataanalytics\hackstat\HackStat (1)\hackstat2k19\Trainset.csv')
print(len(dataset))
print(dataset.head())

10480
   Homepage  Homepage _Duration  Aboutus  Aboutus_Duration  Contactus  \
0       1.0                10.0      0.0               0.0        9.0   
1       2.0                15.0      0.0               0.0       10.0   
2       1.0                85.0      0.0               0.0       14.0   
3       5.0               175.1      0.0               0.0       26.0   
4       2.0                25.0      0.0               0.0        5.0   

   Contactus_Duration  BounceRates  ExitRates  PageValues  SpecialDay Month  \
0          700.000000     0.000000   0.011111   29.621890         0.0   Dec   
1          894.666667     0.000000   0.022222    0.000000         0.0   May   
2          306.500000     0.000000   0.004444    0.000000         0.0   Mar   
3          615.559524     0.000000   0.002083   62.773672         0.0   Sep   
4           40.000000     0.066667   0.083333    0.000000         0.6   May   

   OperatingSystems  Browser  Province  TrafficType        VisitorType  \
0     

In [154]:
dataset.insert(6,'total_duration',dataset[['Homepage _Duration', 'Aboutus_Duration', 'Contactus_Duration']].sum(axis=1))
dataset.insert(6,'total_page',dataset[['Homepage', 'Aboutus', 'Contactus']].sum(axis=1))

In [155]:
dataset.corr(method ='pearson') 

,Homepage,Homepage _Duration,Aboutus,Aboutus_Duration,Contactus,Contactus_Duration,total_page,total_duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Province,TrafficType,Weekend,Revenue
Homepage,1.000000,0.611389,0.370215,0.246413,0.429482,0.387341,0.492237,0.435435,-0.223226,-0.315389,0.097086,-0.098597,-0.010392,-0.021107,-0.008533,-0.040111,0.027863,0.133581
Homepage _Duration,0.611389,1.000000,0.308409,0.219011,0.286207,0.336810,0.325795,0.419636,-0.148655,-0.211546,0.070653,-0.076873,-0.011137,-0.007081,-0.004618,-0.017720,0.015390,0.093813
Aboutus,0.370215,0.308409,1.000000,0.619184,0.367977,0.392663,0.405759,0.440577,-0.114298,-0.160139,0.046355,-0.046694,-0.010058,-0.037440,-0.031935,-0.031719,0.036646,0.098742
Aboutus_Duration,0.246413,0.219011,0.619184,1.000000,0.263663,0.316546,0.286724,0.388120,-0.073204,-0.103648,0.029916,-0.025882,-0.009587,-0.013071,-0.025430,-0.024161,0.022565,0.066694
Contactus,0.429482,0.286207,0.367977,0.263663,1.000000,0.877121,0.997184,0.869788,-0.204605,-0.291842,0.058411,-0.027712,0.003864,-0.004774,-0.037369,-0.038515,0.020233,0.159159
Contactus_Duration,0.387341,0.336810,0.392663,0.316546,0.877121,1.000000,0.877324,0.993683,-0.190078,-0.259852,0.057276,-0.041938,0.003389,-0.002174,-0.032715,-0.032117,0.011794,0.160283
total_page,0.492237,0.325795,0.405759,0.286724,0.997184,0.877324,1.000000,0.875131,-0.214749,-0.306016,0.064348,-0.034492,0.002814,-0.006851,-0.036959,-0.040320,0.022356,0.164807
total_duration,0.435435,0.419636,0.440577,0.388120,0.869788,0.993683,0.875131,1.000000,-0.197151,-0.270497,0.062469,-0.047722,0.001667,-0.003370,-0.032786,-0.033291,0.014066,0.164172
BounceRates,-0.223226,-0.148655,-0.114298,-0.073204,-0.204605,-0.190078,-0.214749,-0.197151,1.000000,0.914073,-0.119656,0.076599,0.022692,-0.009486,-0.004839,0.083842,-0.044975,-0.151224
ExitRates,-0.315389,-0.211546,-0.160139,-0.103648,-0.291842,-0.259852,-0.306016,-0.270497,0.914073,1.000000,-0.174720,0.104793,0.012494,0.000345,-0.006945,0.084516,-0.061315,-0.207305


In [156]:
dataset.Weekend = dataset.Weekend.astype(int)
dataset.loc[dataset['VisitorType'] == "Returning_Visitor", 'VisitorType'] = 1
dataset.loc[dataset['VisitorType'] == "New_Visitor", 'VisitorType'] = 2
dataset.loc[dataset['VisitorType'] == "Other", 'VisitorType'] = 3
dataset.loc[dataset['Month'] == "Jan", 'Month'] = 1
dataset.loc[dataset['Month'] == "Feb", 'Month'] = 2
dataset.loc[dataset['Month'] == "Mar", 'Month'] = 3
dataset.loc[dataset['Month'] == "Apl", 'Month'] = 4
dataset.loc[dataset['Month'] == "May", 'Month'] = 5
dataset.loc[dataset['Month'] == "June", 'Month'] = 6
dataset.loc[dataset['Month'] == "Jul", 'Month'] = 7
dataset.loc[dataset['Month'] == "Aug", 'Month'] = 8
dataset.loc[dataset['Month'] == "Sep", 'Month'] = 9
dataset.loc[dataset['Month'] == "Oct", 'Month'] = 10
dataset.loc[dataset['Month'] == "Nov", 'Month'] = 11
dataset.loc[dataset['Month'] == "Dec", 'Month'] = 12

In [157]:
dataset[dataset < 0] = dataset.mean().axis=1
#dataset[dataset < 0] = 0

In [158]:
dataset.isna().sum()
dataset.isnull().sum()

Homepage              14
Homepage _Duration    14
Aboutus               14
Aboutus_Duration      14
Contactus             14
Contactus_Duration    14
total_page             0
total_duration         0
BounceRates           14
ExitRates             14
PageValues             0
SpecialDay             0
Month                  0
OperatingSystems       0
Browser                0
Province               0
TrafficType            0
VisitorType            0
Weekend                0
Revenue                0
dtype: int64

In [159]:
#dataset = dataset.fillna(dataset.mean())
dataset = dataset.dropna()
print(len(dataset))

10466


In [160]:
dataset = dataset.drop('Province',axis=1)
#dataset = dataset.drop('VisitorType',axis=1)
#dataset = dataset.drop('OperatingSystems',axis=1)
#dataset = dataset.drop('TrafficType',axis=1)
#dataset = dataset.drop('Browser',axis=1)
dataset = dataset.drop('Homepage _Duration',axis=1)
dataset = dataset.drop('Aboutus_Duration',axis=1)
#dataset = dataset.drop('Contactus_Duration',axis=1)
dataset = dataset.drop('Homepage',axis=1)
dataset = dataset.drop('Aboutus',axis=1)
#dataset = dataset.drop('Contactus',axis=1)
dataset = dataset.drop('BounceRates',axis=1)
#dataset = dataset.drop('ExitRates',axis=1)
#dataset = dataset.drop('SpecialDay',axis=1)
#dataset = dataset.drop('Month',axis=1)

In [161]:
x = dataset.iloc[:,0:13]
y = dataset.iloc[:,13]
x_train, x_test, y_train,y_test = train_test_split(x,y,random_state=0, test_size=0.2)

In [162]:
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)
x_test = sc_x.transform(x_test)

In [163]:
xgb_model = xgb.XGBClassifier(learning_rate =0.0047,
    n_estimators=500,
    max_depth = 5,
    min_child_weight = 1,
    gamma=0.01,
    subsample=0.45,
    colsample_bytree=0.9,
    reg_alpha=1,
    objective= 'binary:logistic',
    nthread=4,
    scale_pos_weight=1,
    seed=0)
xgb_model.fit(x_train, y_train)
y_pred = xgb_model.predict(x_test)
print("Testing Accuracy :",accuracy_score(y_test, y_pred))
y_pred = xgb_model.predict(x_train)
print("Training Accuracy :",accuracy_score(y_train, y_pred))

Testing Accuracy : 0.9073543457497613
Training Accuracy : 0.9162685140946011


C:\Users\Acer\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Acer\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [164]:
dataset1 = pd.read_csv("G:\dataanalytics\hackstat\HackStat (1)\hackstat2k19/xtest.csv")

In [165]:
dataset1.Weekend = dataset1.Weekend.astype(int)
dataset1.loc[dataset1['VisitorType'] == "Returning_Visitor", 'VisitorType'] = 1
dataset1.loc[dataset1['VisitorType'] == "New_Visitor", 'VisitorType'] = 2
dataset1.loc[dataset1['VisitorType'] == "Other", 'VisitorType'] = 3
dataset1.loc[dataset1['Month'] == "Jan", 'Month'] = 1
dataset1.loc[dataset1['Month'] == "Feb", 'Month'] = 2
dataset1.loc[dataset1['Month'] == "Mar", 'Month'] = 3
dataset1.loc[dataset1['Month'] == "Apl", 'Month'] = 4
dataset1.loc[dataset1['Month'] == "May", 'Month'] = 5
dataset1.loc[dataset1['Month'] == "June", 'Month'] = 6
dataset1.loc[dataset1['Month'] == "Jul", 'Month'] = 7
dataset1.loc[dataset1['Month'] == "Aug", 'Month'] = 8
dataset1.loc[dataset1['Month'] == "Sep", 'Month'] = 9
dataset1.loc[dataset1['Month'] == "Oct", 'Month'] = 10
dataset1.loc[dataset1['Month'] == "Nov", 'Month'] = 11
dataset1.loc[dataset1['Month'] == "Dec", 'Month'] = 12

In [166]:
dataset1.insert(6,'total_duration',dataset1[['Homepage _Duration', 'Aboutus_Duration', 'Contactus_Duration']].sum(axis=1))
dataset1.insert(6,'total_page',dataset1[['Homepage', 'Aboutus', 'Contactus']].sum(axis=1))

In [167]:
dataset1[dataset1 < 0] = dataset1.mean().axis=1
#dataset1[dataset1 < 0] = 0
dataset1 = dataset1.dropna()
#dataset1 = dataset1.fillna(dataset.mean())
print(len(dataset1))

1850


In [168]:
dataset1 = dataset1.drop('Province',axis=1)
#dataset = dataset.drop('VisitorType',axis=1)
#dataset1 = dataset1.drop('OperatingSystems',axis=1)
#dataset1 = dataset1.drop('TrafficType',axis=1)
#dataset1 = dataset1.drop('Browser',axis=1)
dataset1 = dataset1.drop('BounceRates',axis=1)
#dataset1 = dataset1.drop('ExitRates',axis=1)
#dataset1 = dataset1.drop('SpecialDay',axis=1)
dataset1 = dataset1.drop('Homepage _Duration',axis=1)
dataset1 = dataset1.drop('Aboutus_Duration',axis=1)
dataset1 = dataset1.drop('Homepage',axis=1)
dataset1 = dataset1.drop('Aboutus',axis=1)
#dataset1 = dataset1.drop('Month',axis=1)
#dataset1 = dataset1.drop('Contactus',axis=1)

In [169]:
print(dataset1.head())

   ID  Contactus  total_page  total_duration  Contactus_Duration  ExitRates  \
0   1          1           1        0.000000            0.000000   0.200000   
1   2         24          24      743.750000          743.750000   0.016667   
2   3         13          13      129.500000          129.500000   0.053846   
3   4         59          62     1804.422222         1720.422222   0.038232   
4   5         13          13      390.500000          390.500000   0.053846   

   PageValues  SpecialDay  Month  OperatingSystems  Browser  TrafficType  \
0         0.0         0.0      5                 1        1            1   
1         0.0         0.0     12                 2        2            2   
2         0.0         0.0     11                 2        2            1   
3         0.0         0.0     10                 3        2            3   
4         0.0         0.0     12                 2        2           13   

   VisitorType  Weekend  
0            1        0  
1            2  

In [170]:
x = dataset1.iloc[:,0:13]
#y = dataset1.iloc[:,13]

In [171]:
x = sc_x.transform(x)

In [172]:
y_pred = xgb_model.predict(x)
#cm = confusion_matrix(y, y_pred)
#print(cm)
#print(accuracy_score(y, y_pred))
#print(classification_report(y,y_pred))

C:\Users\Acer\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [173]:
y_pred=pd.DataFrame(y_pred)

In [174]:
y_pred.to_csv("submision.csv",index=True)